In [1]:
import pymesh
import numpy as np
import random
import bisect
import math
from sklearn.metrics import pairwise_distances
from scipy.spatial import distance
from sklearn.metrics.pairwise import euclidean_distances

In [2]:
#load obj files
tea_mesh = pymesh.load_mesh("teapot.obj")
violin_mesh = pymesh.load_mesh("violin_case.obj")

In [3]:
#get vertices and faces
tea_vertices = tea_mesh.vertices
tea_faces = tea_mesh.faces
violin_vertices = violin_mesh.vertices
violin_faces = violin_mesh.faces

In [4]:
#sample points from triangles uniformly
def samplePointsUniformly(area_cumu, faces, vertices):
    P = np.zeros(shape=(10000, 3))
    for i in range(0, 10000):
        r = random.uniform(0, area_cumu[len(area_cumu) - 1])
        r1 = random.uniform(0, 1)
        r2 = random.uniform(0, 1)
        r3 = random.uniform(0, 1)
        tri_index = bisect.bisect_left(area_cumu, r)
        tri = faces[tri_index]
        A = vertices[tri[0]]
        B = vertices[tri[1]]
        C = vertices[tri[2]]
        p = (1 - math.sqrt(r1)) * A + math.sqrt(r1) * (1 - r2) * B + math.sqrt(r1) * r2 * C
        P[i] = p
    return P

In [5]:
#furthest point sampling
def total_distances(p, pts):
    return ((p - pts)**2).sum(axis=1)

def furthestPointSampling(pts, k):
    furthest_pts = np.zeros((k, 3))
    furthest_pts[0] = pts[np.random.randint(len(pts))]
    distances = total_distances(furthest_pts[0], pts)
    for i in range(1, k):
        furthest_pts[i] = pts[np.argmax(distances)]
        distances = np.minimum(distances, total_distances(furthest_pts[i], pts))
    return furthest_pts

In [6]:
# P1 stores 10000 points sampled from the teapot
# D1 is the distance matrix of P1
# S1 stores 1000 points sampled from P1
tea_mesh.add_attribute("face_area")
tea_face_areas = tea_mesh.get_attribute("face_area")
tea_face_areas_cumulative = []
total_area = 0
for i in range(len(tea_face_areas)):
    total_area = total_area + tea_face_areas[i]
    tea_face_areas_cumulative.append(total_area)
P1 = samplePointsUniformly(tea_face_areas_cumulative, tea_faces, tea_vertices)
D1 = pairwise_distances(P1)
S1 = furthestPointSampling(P1, 1000)

In [7]:
# P2 stores 10000 points sampled from the violin case
# D2 is the distance matrix of P2
# S2 stores 1000 points sampled from P2
violin_mesh.add_attribute("face_area")
violin_face_areas = violin_mesh.get_attribute("face_area")
violin_face_areas_cumulative = []
total_area = 0
for i in range(len(violin_face_areas)):
    total_area = total_area + violin_face_areas[i]
    violin_face_areas_cumulative.append(total_area)
P2 = samplePointsUniformly(violin_face_areas_cumulative, violin_faces, violin_vertices)
D2 = pairwise_distances(P2)
S2 = furthestPointSampling(P2, 1000)

In [8]:
#save the S1 to new_teapot.obj
newTri = pymesh.tetgen()
newTri.points = S1
newTri.run()
new_tea = newTri.mesh
pymesh.save_mesh("new_teapot.obj", new_tea, use_float=True)

In [9]:
#save the S2 to new_violin_case.obj
newTri = pymesh.tetgen()
newTri.points = S2
newTri.run()
new_violin = newTri.mesh
pymesh.save_mesh("new_violin_case.obj", new_violin, use_float=True)

In [178]:
#Q3 Implementation of Hungarian Algorithm
# Helper functions for hungarian algorithm
def choice_exist_in_all_covered_col(choice, covered_cols):
    for c, col in enumerate(covered_cols):
        if col:
            if not np.any(choice[:, c]):
                return False
    return True

def mark_cols_have_zeros_in_marked_rows(zeros_location, covered_rows, covered_cols):
    for r in range(len(zeros_location)):
            for c in range(len(zeros_location)):
                if zeros_location[r][c] and covered_rows[r] and not covered_cols[c]:
                    covered_cols[c] = True

def find_col_idx_with_no_choice(choice, covered_cols): 
    for c, col in enumerate(choice.T):
        if covered_cols[c] and not np.any(col):
            return c
    return -1

# find a zero with col_idx_with_no_choice that does not have a row with a choice
def find_row_idx_with_no_choice(zeros_location, choice, col_idx_with_no_choice):
    for i, row in enumerate(choice):
        for j, col in enumerate(row):
            if j == col_idx_with_no_choice and zeros_location[i][j] and not np.any(row):
                return i
    return -1

# find a row idx for choice such that the col to change is optimal
# otherwise, return random row and col
def find_choice_row_col(zeros_location, choice, col_idx_with_no_choice):
    row_indices, = np.where(zeros_location[:, col_idx_with_no_choice])
    for row_idx in row_indices:
        col_indices, = np.where(choice[row_idx])
        col_idx = col_indices[0]
        if find_row_idx_with_no_choice(zeros_location, choice, col_idx) != -1:
            return row_idx, col_idx
    random.shuffle(row_indices)
    col_idx, = np.where(choice[row_indices[0]])
    return row_indices[0], col_idx[0]

def cover_zeros(d):
    zeros_location = (d == 0)
    choice = np.zeros(np.shape(d), dtype=bool)
    # Step 3 cover all zeros using minimum number of lines
    covered_rows = [False] * np.shape(d)[0]
    covered_cols = [False] * np.shape(d)[1]
    
    while True:
        covered_rows = [False] * np.shape(d)[0]
        covered_cols = [False] * np.shape(d)[1]
        
        # mark all rows in which no choice has been made
        for r, row in enumerate(choice):
            if not np.any(row):
                covered_rows[r] = True
        
        # if no marked row left
        if not np.sum(covered_rows):
#             print(1)
            return covered_rows, covered_cols, choice

        # mark all cols not already marked which have zeros in marked rows        
        before = np.sum(covered_cols)
        mark_cols_have_zeros_in_marked_rows(zeros_location, covered_rows, covered_cols)
        
        # if no new marked cols
        if before == np.sum(covered_cols):
#             print(2)
            return covered_rows, covered_cols, choice
        
        # while there is some choice in every marked col
        while choice_exist_in_all_covered_col(choice, covered_cols):
            # mark all rows not already marked that have choices in marked col
            before = np.sum(covered_rows)
            for r in range(len(choice)):
                for c in range(len(choice)):
                    if covered_cols[c] and choice[r][c] and not covered_rows[r]:
                        covered_rows[r] = True
            
            if before == np.sum(covered_rows):
#                 print(3)
                return covered_rows, covered_cols, choice
            
            before = np.sum(covered_cols)
            # mark all cols not already marked which have zeros in marked rows
            mark_cols_have_zeros_in_marked_rows(zeros_location, covered_rows, covered_cols)
            
            if before == np.sum(covered_cols):
#                 print(4)
                return covered_rows, covered_cols, choice
        
        # find a marked col that doesn't have a choice
        col_idx_with_no_choice = find_col_idx_with_no_choice(choice, covered_cols)
        while col_idx_with_no_choice != -1:
            # find a zero with col_idx_with_no_choice that does not have a row with a choice
            choice_row_idx = find_row_idx_with_no_choice(zeros_location, choice, col_idx_with_no_choice)
            
            choice_col_idx = -1
            if choice_row_idx == -1:
                # find a row to swap
                choice_row_idx, choice_col_idx = find_choice_row_col(zeros_location, choice, col_idx_with_no_choice)
                
                choice[choice_row_idx, choice_col_idx] = False
                
            choice[choice_row_idx, col_idx_with_no_choice] = True
            
            col_idx_with_no_choice = choice_col_idx

def hungarian(d):
    # Step 1:
    # Row reduction
    for i, row in enumerate(d):
        d[i] -= np.min(row)
    # Step 2:
    # Column reduction
    for i, col in enumerate(d.T):
            d[:, i] -= np.min(col)
        
    
    while True:
        # Step 3 cover all zeros using minimum number of lines
        covered_rows, covered_cols, choice = cover_zeros(d)
        lines = np.sum(covered_cols) + len(covered_rows) - np.sum(covered_rows)
        # step 4 test for optimality
        if lines >= np.shape(d)[0]:
            return choice
        else:
            # Step 5 
            # Determine the smallest entry not covered by any line. 
            # Subtract this entry from each uncovered row, 
            # and then add it to each covered column. 
            # Return to step 3
            covered_rows = np.invert(covered_rows)
            
            min_entry_not_covered = float("inf")
            for r in range(np.shape(d)[0]):
                if covered_rows[r]:
                    continue
                for c in range(np.shape(d)[1]):
                    if covered_cols[c]:
                        continue
                    if d[r][c] < min_entry_not_covered:
                        min_entry_not_covered = d[r][c]
            # print(min_entry_not_covered)
            for r, row in enumerate(d):
                if not covered_rows[r]:
                    d[r] -= min_entry_not_covered
            for c, col in enumerate(d.T):
                if covered_cols[c]:
                    d[:, c] += min_entry_not_covered
            # back to step 3 again
            # cover_zeros(d)

In [205]:
from sklearn.metrics.pairwise import euclidean_distances

# sample just 500 points from previous meshes
S1_500 = furthestPointSampling(P1, 100)
S2_500 = furthestPointSampling(P2, 100)

# computes distance matrix between S1_500 and S2_500
d_origin = euclidean_distances(S1_500, S2_500)
d = d_origin.copy()

In [208]:
# emd computed by own implementation
mask = hungarian(d)
emd = np.sum(d_origin * mask)
print(emd)

5286.315963211391


In [210]:
# emd computed by scipy
from scipy.optimize import linear_sum_assignment

cost = d_origin.copy()
row_ind, col_ind = linear_sum_assignment(cost)
print(cost[row_ind, col_ind].sum())

5286.315963211392
